Implementing uncertainty quantification with bootstrapping on a toy dataset

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

2025-03-13 12:09:49.839071: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 12:09:50.329554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-13 12:09:50.329608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-13 12:09:50.434135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 12:09:50.595295: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# check GPU availability
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Set random seed for reproducibility
np.random.seed(21)
tf.random.set_seed(21)

In [4]:
# Step 1: Generate random data
n_samples = 1000
n_features = 10

In [5]:
# Generate random input features (X) and output (y)
X = np.random.randn(n_samples, n_features)
y = 2 * np.sum(X, axis=1) + np.random.randn(n_samples)  # Simple linear relation + noise

In [6]:
X.shape, y.shape

((1000, 10), (1000,))

In [7]:
# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((800, 10), (800,), (100, 10), (100,), (100, 10), (100,))

In [9]:
# Step 2: Define a simple fully connected neural network for Phase I (bootstrap models)
def create_bootstrap_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [10]:
# Step 3: Define the noise variance estimation network (Phase II)
def create_noise_variance_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for variance prediction
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [11]:
# I think upto this moment it's clear, the only thing is for the second network, it is not using the correct loss function which is based on the MLE, and we need to adjust that. Let's figure out in the below code blocks if this is computing the r_2 values for the secnd network correctly.

In [12]:
# careful attention to this function, is this doing teh correct thing?

In [13]:
X_train.shape[1:]

(10,)

In [14]:
# Step 4: Train bootstrap neural networks (Phase I)
def train_bootstrap_nns(X_train, y_train, B=10):
    bootstrap_models = []
    bootstrap_predictions = []

    # Generate B bootstrap samples and train models
    for _ in range(B):
        # Create a bootstrap sample by sampling with replacement
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        # Create and train a new model
        model = create_bootstrap_nn(X_train.shape[1:])
        model.fit(X_bootstrap, y_bootstrap, epochs=50, batch_size=32, verbose = 0)
        bootstrap_models.append(model)

        # Store predictions on the original training data
        predictions = model.predict(X_train)
        bootstrap_predictions.append(predictions)

    return bootstrap_models, np.array(bootstrap_predictions)

In [15]:
# The function does the bootstraps, and save both the models from the bootstraps - for future, for the test set evaluations, and the predictions for the train set as those are necessary for the second network

In [16]:
# Okay, now need to verify if the model variance that is computed straight away using np.var and the one specified in the paper by equation 6 are the same thing. If they are, then most of the problems in the code here are solved.

In [17]:
# Step 6: Train the models
# Train the bootstrap models (Phase I)
bootstrap_models, bootstrap_predictions = train_bootstrap_nns(X_train, y_train, B=10)

2025-03-13 12:10:08.593864: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2025-03-13 12:10:11.060259: I external/local_xla/xla/service/service.cc:168] XLA service 0x14d3b9b18900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-13 12:10:11.060296: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2025-03-13 12:10:11.126397: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-13 12:10:11.276683: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1741885811.575417  191301 device_compiler.h:186] Compiled cluster using XLA!  Thi

25/25 [==============================] - 0s 1ms/step


In [18]:
# variances all at once
all_vars_at_once = np.var(bootstrap_predictions, axis=0)

In [19]:
all_vars_at_once.shape

(800, 1)

In [20]:
# all_vars_at_once

In [21]:
bootstrap_predictions.shape

(10, 800, 1)

In [22]:
# function to get the variances according to the equation 6 in the paper
def equation_6_model_variance(all_preds):
    all_vars = []
    for i in range(all_preds.shape[1]):
        var = (1/(all_preds.shape[0] ) - 1)*np.sum(np.square(all_preds[:,i,:] - np.mean(all_preds[:,i,:])))
        all_vars.append(var)

    return np.array(all_vars)
    

In [23]:
# Note that this is the model variance
all_vars_eq_6 = equation_6_model_variance(bootstrap_predictions)

In [37]:
all_vars_eq_6.shape

(800,)

In [43]:
bootstrap_predictions.shape

(10, 800, 1)

In [46]:
y_train.shape, bootstrap_predictions[0].shape

((800,), (800, 1))

In [49]:
((y_train - bootstrap_predictions[0])**2).shape

(800, 800)

In [61]:
((y_train - bootstrap_predictions[0].reshape(-1))**2).shape

(800,)

In [25]:
# Okay, found the discripancy, in finding the variance with np.var, the division is with B and not with B-1 - Therefore, we need to have this custom equation in order to get the correct variance associated with the bootstrapped values. 

In [26]:
# Okay, now we need to find a way to include this variance inside the work that we are doing

In [27]:
# Step 5: Phase II - Train the noise variance estimation network (NNₑ)
def compute_r_squared(y_true, y_pred, model_variance):
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

In [63]:
# Edit the function to include the correct model misspecification varaince
def train_noise_variance_nn(X_train, y_train, bootstrap_predictions, model_variance, B=10):
    # Calculate r^2(x_i) for each bootstrap model
    residuals_all = []
    for b in range(B):
        # need to reshape the prediction vector as it has shape (800,1) and the train y shape is just (800,)
        residuals = compute_r_squared(y_train, bootstrap_predictions[b].reshape(-1), model_variance)
        residuals_all.append(residuals)
    
    residuals_all = np.stack(residuals_all, axis=-1)
    mean_residuals = np.mean(residuals_all, axis=-1)  # Averaging residuals across bootstrap models

    # Create and train the NNₑ for noise variance estimation
    nn_e = create_noise_variance_nn(X_train.shape[1:])
    nn_e.fit(X_train, mean_residuals, epochs=50, batch_size=32)

    return nn_e

In [64]:
# Train the noise variance estimation model (Phase II)
nn_e = train_noise_variance_nn(X_train, y_train, bootstrap_predictions, all_vars_eq_6, B=10)

Epoch 1/50
25/25 [==============================] - 1s 2ms/step - loss: 7.1100
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 4.2758
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 3.8012
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: 3.6134
Epoch 5/50
25/25 [==============================] - 0s 2ms/step - loss: 3.5103
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 3.4422
Epoch 7/50
25/25 [==============================] - 0s 2ms/step - loss: 3.3883
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 3.3339
Epoch 9/50
25/25 [==============================] - 0s 1ms/step - loss: 3.2983
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 3.2588
Epoch 11/50
25/25 [==============================] - 0s 3ms/step - loss: 3.2316
Epoch 12/50
25/25 [==============================] - 0s 3ms/step - loss: 3.1965
Epoch 13/50
25/25 [==============================

In [65]:
# Step 7: Evaluate the models on the test set
# Make predictions with the bootstrap models
bootstrap_preds_test = np.array([model.predict(X_test) for model in bootstrap_models])

4/4 [==============================] - 0s 1ms/step


In [66]:
bootstrap_preds_test.shape

(10, 100, 1)

In [67]:
# Calculate the mean prediction across all bootstrap models
bootstrap_mean_preds_test = np.mean(bootstrap_preds_test, axis=0)

In [68]:
bootstrap_mean_preds_test.shape

(100, 1)

In [69]:
# Estimate the noise variance using NNₑ
predicted_variance_test = nn_e.predict(X_test)

4/4 [==============================] - 0s 2ms/step


In [70]:
predicted_variance_test.shape

(100, 1)

In [71]:
# Print results
print(f"Mean prediction for test set: {np.mean(bootstrap_mean_preds_test)}")
print(f"Estimated noise variance (NNₑ) for test set: {np.mean(predicted_variance_test)}")

Mean prediction for test set: -0.17154209315776825
Estimated noise variance (NNₑ) for test set: 2.406175374984741


But, notice that this method has a mishap. We need to use the loss with MLE for the second NN for the error variance, but this model is still using the same loss (mse) as the NN used for the bootsraps. We will look at this stpe in the next notbeook.